In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2
import random
import os
from PIL import Image

In [13]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import cv2
import json
import os
import time #for tracking
from torch.utils.data import Dataset, DataLoader

class chinPointDataset(Dataset):
    def __init__(self, imageFolder, annotFile, idealWidth, idealHeight):
        self.imageFolder = imageFolder
        self.annotFile = annotFile
        self.idealWidth = idealWidth
        self.idealHeight = idealHeight
        t1=time.time()
        # count images
        self.size = 0
        for image in os.listdir(imageFolder):
            if image.endswith(".png"): #redundant
                self.size += 1
        
        # open json
        with open(annotFile, 'r') as f:
            self.jsonAnnot = json.load(f)
        
        # preallocate arrays
        self.allImgsArray = np.zeros([self.size, idealHeight, idealWidth], dtype=np.float32)
        self.allAnnotsArray = np.zeros([self.size, 2], dtype=np.float32)
        
        # load all images and annotations

        # loop through and load all iamges in a folder. 
        #create a numpy array from these images.
        #then recolor it and resize it and app
        t2=time.time()
        for idx, image in enumerate(os.listdir(imageFolder)):
            if image.endswith(".png"): #redundant
                
                imgArr = cv2.imread(os.path.join(imageFolder, image))
                height, width, channels = imgArr.shape
                imgArr = cv2.cvtColor(imgArr, cv2.COLOR_BGR2GRAY)
                imgArr = cv2.resize(imgArr, (idealWidth, idealHeight))  # cv2.resize takes (width, height)
                
                # normalize to 0-1 - crucial because nns generally work best with 0-1 normalized data... 
                self.allImgsArray[idx] = imgArr.astype(np.float32) / 255.0
                
                # scale annotation from original coords to ideal coords
                origX, origY = self.jsonAnnot[image]
                
                self.allAnnotsArray[idx] = [
                    origX * (idealWidth / width),
                    origY * (idealHeight / height)
                ]
        t3=time.time()
        print(f"Time taken for each stage:")
        print(f"Time taken for counting & loading annotations: {t2-t1} seconds")
        print(f"Time taken for loading & processing images: {t3-t2} seconds")
        print(f"Total time taken: {t3-t1} seconds")
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        # returns (image, annotation) as tensors
        img = torch.from_numpy(self.allImgsArray[idx]).unsqueeze(0)  # shape: [1, H, W] for single channel
        annot = torch.from_numpy(self.allAnnotsArray[idx])  # shape: [2]
        return img, annot

imageFolder = "./Imgs"
annotFile = "Imgs_Annotate.json"
idealWidth = 96
idealHeight = 96
chinPointDataset = chinPointDataset(imageFolder, annotFile, idealWidth, idealHeight)

Time taken for each stage:
Time taken for counting & loading annotations: 0.0019998550415039062 seconds
Time taken for loading & processing images: 9.512271881103516 seconds
Total time taken: 9.51427173614502 seconds


In [14]:
chinPointDataset.getitem()

AttributeError: 'chinPointDataset' object has no attribute 'getitem'

In [18]:
d = {"a": 1, "b": 2}

# === ACCESS ===
d["a"]                    # → 1 (raises KeyError if missing)


1

In [22]:
print(d.get("c")    )            # → 1 (returns None if missing)


None


In [23]:
d.get("z",-1)    # → -1 (returns default if missing)

-1

In [24]:
d


{'a': 1, 'b': 2}

In [25]:
d.pop("b")

2

In [27]:
d

{'a': 1}

In [29]:
d[]"b")

AttributeError: 'dict' object has no attribute 'insert'

In [ ]:
#How dictionaries work in python:

#d.update() #overrides values and inserts new ones whereever they do not yet exist. Does it do this regardless of order?
dict = {"a":1, "b":2,"ca":4}
dict.get() #returns none if key is DNE, else its value.
dict.get("item",returnvalue) #returns returnvalue if key is DNE, else the key's value.
#enumerate() splits a list or dict into an index and the value.. by default for a dictionary this results in, if idx,item is done, the item being the key.


In [45]:
print(dict.get("ca"))

4


In [49]:
d.get('z', 1.3**7)

6.274851700000002

In [62]:
l=[5,6,44,1]
for idx,f in enumerate(dict):
    print(idx," and: " + f)

0  and: a
1  and: b
2  and: ca
